# KPI Visualizer

Creates overview KPI .xls by data manipulation of available monthly data points.

## Data processing

### Import and init modules:

In [13]:
import os
import openpyxl
from openpyxl.chart import ScatterChart, LineChart, Reference, Series
from openpyxl.utils.dataframe import dataframe_to_rows

import datascience as ds
from datascience.predicates import are
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

### List available KPI data points

In [14]:
files = os.listdir()
monthly_kpi = [f for f in files if f.startswith('WKPL_KPI_')]
monthly_kpi.sort()
kpi_count = len(monthly_kpi)

data_types = ['Contacts', 'NSL', 'NSL Prof', 'Alert', 
              'Sent (M)', 'Open (M)', 'Click (M)', 'Form (M)', 
              'Unsub (M)', 'Not Opened 10+ (Y)', 'Cookie Linked (A)']

### Create different views

#### Create excel file to store data

In [15]:
excel = openpyxl.Workbook()
source_sheet = excel.active
source_sheet.title = 'Source Data'
for data_type in data_types:
    excel.create_sheet(title=data_type)

#### Storing source data

In [16]:
kpi_dates = []
for csv_path in monthly_kpi:
    # Get the MM-YY date from the path
    kpi_date = csv_path.split('_')[-1][:-4]
    kpi_dates.append(kpi_date)
    # Load KPI table
    kpi_csv = ds.Table.read_table(csv_path)
    kpi_csv.relabel('Segment', kpi_date)
    # Save all data points to source sheet
    df = kpi_csv.to_df()
    for row in dataframe_to_rows(df, index=False, header=True):
        source_sheet.append(row)
    # Add spacer
    source_sheet.append([])

#### Creating data tables

In [17]:
for data_type in data_types:
    # Creation of a data table for each data_type
    data_table = ds.Table(ds.make_array(
            data_type,
            'ALL + WMR', 'ALL - WMR',
            'LEG + WMR', 'LEG - WMR',
            'PUB + WMR', 'PUB - WMR',
            'FIR + WMR', 'FIR - WMR',
            'OTH + WMR', 'OTH - WMR'))
    
    # Switching to correct excel sheet
    data_sheet = excel[data_type]
    data_sheet.freeze_panes = 'B2'
    
    for csv_path in monthly_kpi:
        # Get the MM-YY date from the path
        kpi_date = csv_path.split('_')[-1][:-4]
        # Load KPI table
        kpi_csv = ds.Table.read_table(csv_path)
        # Create row with data for each month
        data_row = [kpi_date]
        data_row.extend(kpi_csv.column(data_type))
        # Save all data points to table
        data_table.append(data_row)
        
    # Saves data to excel file
    df = data_table.to_df()
    for row in dataframe_to_rows(df, index=False, header=True):
        data_sheet.append(row)
        
    # Create chart for whole segment groups + and - WMR
    sc_all = ScatterChart()
    sc_all.title = data_type + ' Change'
    sc_all.style = 13
    sc_all.y_axis.title = data_type
    sc_all.x_axis.title = 'Month'

    x_values = Reference(data_sheet, min_col=1, min_row=1, max_row=(kpi_count + 1))
    for i in range(2, 4):
        y_values = Reference(data_sheet, min_col=i, min_row=1, max_row=(kpi_count + 1))
        series = Series(y_values, x_values, title_from_data=True)
        sc_all.series.append(series)

    data_sheet.add_chart(sc_all, 'M4')

    # Create chart for all +WMR segments 
    sc_seg = ScatterChart()
    sc_seg.title = data_type + ' Change per Segment (WMR)'
    sc_seg.style = 13
    sc_seg.y_axis.title = data_type
    sc_seg.x_axis.title = 'Month'

    x_values = Reference(data_sheet, min_col=1, min_row=1, max_row=(kpi_count + 1))
    for i in range(4, 12, 2):
        y_values = Reference(data_sheet, min_col=i, min_row=1, max_row=(kpi_count + 1))
        series = Series(y_values, x_values, title_from_data=True)
        sc_seg.series.append(series)

    data_sheet.add_chart(sc_seg, 'M20')
    
    # Create chart for all -WMR segments 
    sc_seg2 = ScatterChart()
    sc_seg2.title = data_type + ' Change per Segment (no WMR)'
    sc_seg2.style = 13
    sc_seg2.y_axis.title = data_type
    sc_seg2.x_axis.title = 'Month'

    x_values = Reference(data_sheet, min_col=1, min_row=1, max_row=(kpi_count + 1))
    for i in range(5, 12, 2):
        y_values = Reference(data_sheet, min_col=i, min_row=1, max_row=(kpi_count + 1))
        series = Series(y_values, x_values, title_from_data=True)
        sc_seg2.series.append(series)

    data_sheet.add_chart(sc_seg2, 'M36')

## Saving data to excel

In [18]:
excel.save('WKPL_Contact_KPI.xlsx')